# General alignment implementation

In [1]:
import numpy as np
import pandas as pd

In [2]:
def create_scoring_matrix(S_raw, rows_string, cols_string):
    df = pd.DataFrame(S_raw)
    df.index = rows_string.split()
    df.columns = cols_string.split()
    return df


def general_alignment(S, reference, read, kind):
    X = '_' + reference
    Y = '_' + read
    m = len(X)
    n = len(Y)
    
    D = np.zeros((m, n), dtype=int)
    T = np.zeros((m, n), dtype=str)
    
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            if i == 0 and j == 0:
                D[i, j] = 0
                T[i, j] = ''
            
            elif i == 0:
                if kind == 'ESFV':
                    D[i, j] = 0
                    T[i, j] = 'i'
                else:
                    D[i, j] = D[i, j-1] + S.loc[x][y]
                    T[i, j] = 'I'
            
            elif j == 0:
                if kind == 'ESFV':
                    D[i, j] = 0
                    T[i, j] = 'd'
                else:
                    D[i, j] = D[i-1, j] + S.loc[x][y]
                    T[i, j] = 'D'
            
            else:
                edits = []
                
                match_replace = D[i-1, j-1] + S.loc[x][y]
                edits.append('M' if x == y else 'R')
                    
                if kind == 'ESFV' and j == n-1:
                    deletion = D[i-1, j]
                    edits.append('d')
                else:
                    deletion = D[i-1, j] + S.loc[x]['_']
                    edits.append('D')
                
                if kind == 'ESFV' and i == m-1:
                    insertion = D[i, j-1]
                    edits.append('i')
                else:
                    insertion = D[i, j-1] + S.loc['_'][y]
                    edits.append('I')
                
                actions = np.asarray([match_replace, deletion, insertion])
                
                D[i, j] = max(actions)
                T[i, j] = edits[actions.argmax()]
                
    df_d = pd.DataFrame(D, list(X), list(Y))
    df_t = pd.DataFrame(T, list(X), list(Y))
    
    ret_t = ''
    ret_x = ''
    ret_bars = ''
    ret_y = ''
    
    # i, j = m, n
    
    while True:
        if T[i, j] == '':
            break
                
        if T[i, j] in ['M', 'R']:
            ret_t += T[i, j]
            ret_x += X[i]
            ret_y += Y[j]
            ret_bars += '|' if T[i, j] == 'M' else ' '
            i -= 1
            j -= 1
        elif T[i, j] == 'D':
            ret_t += T[i, j]
            ret_x += X[i]
            ret_y += '_'
            ret_bars += ' '
            i -= 1
        elif T[i, j] == 'I':
            ret_t += T[i, j]
            ret_x += '_'
            ret_y += Y[j]
            ret_bars += ' '
            j -= 1
        elif T[i, j] == 'd':
            ret_t += ' '
            ret_x += X[i]
            ret_y += ' '
            ret_bars += ' '
            i -= 1
        elif T[i, j] == 'i':
            ret_t += ' '
            ret_x += ' '
            ret_y += Y[j]
            ret_bars += ' '
            j -= 1
        
    alignment = '{x}\n{bars}\n{t}\n{bars}\n{y}'.format(x=ret_x[::-1],
                                                       bars=ret_bars[::-1],
                                                       t=ret_t[::-1],
                                                       y=ret_y[::-1])
    
    return df_d, df_t, str(alignment)

## Edit distance
Levenshtein distance

In [3]:
S_raw = [[0, -1, -1, -1, -1],
         [-1, 0, -1, -1, -1],
         [-1, -1, 0, -1, -1],
         [-1, -1, -1, 0, -1],
         [-1, -1, -1, -1, np.NaN]]

S = create_scoring_matrix(S_raw, 'A C G T _', 'A C G T _')

In [4]:
reference = 'GCGTATGCACGC'
read = 'GCTATGCCACGC'

D, T, alignment = general_alignment(S, reference, read, 'ED')

print(alignment)
D
T

GCGTATG_CACGC
|| |||| |||||
MMDMMMMIMMMMM
|| |||| |||||
GC_TATGCCACGC


,_,G,C,T,A,T,G,C,C,A,C,G,C
_,,I,I,I,I,I,I,I,I,I,I,I,I
G,D,M,I,I,I,I,M,I,I,I,I,M,I
C,D,D,M,I,I,I,I,M,M,I,M,I,M
G,D,M,D,R,R,R,M,I,I,I,I,M,I
T,D,D,D,M,R,M,I,R,R,R,R,R,R
A,D,D,D,D,M,I,R,R,R,M,I,I,I
T,D,D,D,M,D,M,I,I,I,I,R,R,R
G,D,M,D,D,D,D,M,I,I,I,I,M,I
C,D,D,M,D,D,D,D,M,M,I,M,I,M
A,D,D,D,D,M,D,D,D,R,M,I,I,I


## Global alignment
Needleman-Wunsch alignment

In [5]:
S_raw = [[1, -3, -1, -3, -7],
         [-3, 1, -3, -1, -7],
         [-1, -3, 1, -3, -7],
         [-3, -1, -3, 1, -7],
         [-7, -7, -7, -7, np.NaN]]

S = create_scoring_matrix(S_raw, 'A C G T _', 'A C G T _')
S

,A,C,G,T,_
A,1,-3,-1,-3,-7.0
C,-3,1,-3,-1,-7.0
G,-1,-3,1,-3,-7.0
T,-3,-1,-3,1,-7.0
_,-7,-7,-7,-7,NaN


In [6]:
reference = 'TACGTCAGC'
read = 'TATGTCATGC'

D, T, alignment = general_alignment(S, reference, read, 'GA')

print(alignment)
D

TACGTCA_GC
|| |||| ||
MMRMMMMIMM
|| |||| ||
TATGTCATGC


,_,T,A,T,G,T,C,A,T,G,C
_,0,-7,-14,-21,-28,-35,-42,-49,-56,-63,-70
T,-7,1,-6,-13,-20,-27,-34,-41,-48,-55,-62
A,-14,-6,2,-5,-12,-19,-26,-33,-40,-47,-54
C,-21,-13,-5,1,-6,-13,-18,-25,-32,-39,-46
G,-28,-20,-12,-6,2,-5,-12,-19,-26,-31,-38
T,-35,-27,-19,-11,-5,3,-4,-11,-18,-25,-32
C,-42,-34,-26,-18,-12,-4,4,-3,-10,-17,-24
A,-49,-41,-33,-25,-19,-11,-3,5,-2,-9,-16
G,-56,-48,-40,-32,-24,-18,-10,-2,2,-1,-8
C,-63,-55,-47,-39,-31,-25,-17,-9,-3,-1,0


In [7]:
reference = 'TAGCGTC'
read = 'TATGTC'

D, T, alignment = general_alignment(S, reference, read, 'GA')

print(alignment)

TAGCGTC
||  |||
MMDRMMM
||  |||
TA_TGTC


In [8]:
reference = 'TAGCGTC'
read = 'TAAGTC'

D, T, alignment = general_alignment(S, reference, read, 'GA')

print(alignment)

TAGCGTC
||  |||
MMRDMMM
||  |||
TAA_GTC


## Longest common subsequence

In [9]:
S_raw = [[1, -1, -1, -1, 0],
         [-1, 1, -1, -1, 0],
         [-1, -1, 1, -1, 0],
         [-1, -1, -1, 1, 0],
         [0, 0, 0, 0, np.NaN]]

S = create_scoring_matrix(S_raw, 'A C G T _', 'A C G T _')
S

,A,C,G,T,_
A,1,-1,-1,-1,0.0
C,-1,1,-1,-1,0.0
G,-1,-1,1,-1,0.0
T,-1,-1,-1,1,0.0
_,0,0,0,0,NaN


In [10]:
reference = 'TACGTCAGA'
read = 'AAAGTCATGC'

D, T, alignment = general_alignment(S, reference, read, 'LCS')

print(alignment)
D

__TACGTCA_G_A
   | |||| |  
IIDMDMMMMIMID
   | |||| |  
AA_A_GTCATGC_


,_,A,A,A,G,T,C,A,T,G,C
_,0,0,0,0,0,0,0,0,0,0,0
T,0,0,0,0,0,1,1,1,1,1,1
A,0,1,1,1,1,1,1,2,2,2,2
C,0,1,1,1,1,1,2,2,2,2,3
G,0,1,1,1,2,2,2,2,2,3,3
T,0,1,1,1,2,3,3,3,3,3,3
C,0,1,1,1,2,3,4,4,4,4,4
A,0,1,2,2,2,3,4,5,5,5,5
G,0,1,2,2,3,3,4,5,5,6,6
A,0,1,2,3,3,3,4,5,5,6,6


## End space-free variant

In [11]:
S_raw = [[1, -3, -1, -3, -7],
         [-3, 1, -3, -1, -7],
         [-1, -3, 1, -3, -7],
         [-3, -1, -3, 1, -7],
         [-7, -7, -7, -7, np.NaN]]

S = create_scoring_matrix(S_raw, 'A C G T _', 'A C G T _')
S

,A,C,G,T,_
A,1,-3,-1,-3,-7.0
C,-3,1,-3,-1,-7.0
G,-1,-3,1,-3,-7.0
T,-3,-1,-3,1,-7.0
_,-7,-7,-7,-7,NaN


In [12]:
reference = 'TACGTCAGA'
read = 'GTC'

D, T, alignment = general_alignment(S, reference, read, 'ESFV')

print(alignment)
D
T

TACGTCAGA
   |||   
   MMM   
   |||   
   GTC   


,_,G,T,C
_,,i,i,i
T,d,R,M,d
A,d,R,R,d
C,d,R,R,d
G,d,M,R,d
T,d,R,M,d
C,d,R,R,M
A,d,R,R,d
G,d,M,R,d
A,d,i,i,d


## IUPAC Nucleotide representations
https://en.wikipedia.org/wiki/Nucleic_acid_notation

In [13]:
S_raw = [[1, -3, -1, -3, 1, -7],
         [-3, 1, -3, -1, 1, -7],
         [-1, -3, 1, -3, 1, -7],
         [-3, -1, -3, 1, 1, -7],
         [1, 1, 1, 1, 1, -7],
         [-7, -7, -7, -7, -7, np.NaN]]

S = create_scoring_matrix(S_raw, 'A C G T N _', 'A C G T N _')
S

,A,C,G,T,N,_
A,1,-3,-1,-3,1,-7.0
C,-3,1,-3,-1,1,-7.0
G,-1,-3,1,-3,1,-7.0
T,-3,-1,-3,1,1,-7.0
N,1,1,1,1,1,-7.0
_,-7,-7,-7,-7,-7,NaN


In [14]:
S_raw = [[1, -3, -1, -3, -7],
         [-3, 1, -3, -1, -7],
         [-1, -3, 1, -3, -7],
         [-3, -1, -3, 1, -7],
         [1, 1, 1, 1, -7],
         [-7, -7, -7, -7, np.NaN]]

S = create_scoring_matrix(S_raw, 'A C G T N _', 'A C G T _')
S

,A,C,G,T,_
A,1,-3,-1,-3,-7.0
C,-3,1,-3,-1,-7.0
G,-1,-3,1,-3,-7.0
T,-3,-1,-3,1,-7.0
N,1,1,1,1,-7.0
_,-7,-7,-7,-7,NaN


In [15]:
S.loc['N','T']

1

In [16]:
reference = 'TACGNCAGA'
read = 'GTC'

D, T, alignment = general_alignment(S, reference, read, 'ESFV')

print(alignment)
D

TACGNCAGA
   | |   
   MRM   
   | |   
   GTC   


,_,G,T,C
_,0,0,0,0
T,0,-3,1,0
A,0,-1,-6,0
C,0,-3,-2,0
G,0,1,-6,0
N,0,1,2,0
C,0,-3,0,3
A,0,-1,-6,3
G,0,1,-4,3
A,0,0,0,3


In [17]:
reference = 'TACGNCAGA'
read = 'GTC'

D, T, alignment = general_alignment(S, reference, read, 'GA')

print(alignment)
D

TACGNCAGA
   | |   
DDDMRMDDD
   | |   
___GTC___


,_,G,T,C
_,0,-7,-14,-21
T,-7,-3,-6,-13
A,-14,-8,-6,-9
C,-21,-15,-9,-5
G,-28,-20,-16,-12
N,-35,-27,-19,-15
C,-42,-34,-26,-18
A,-49,-41,-33,-25
G,-56,-48,-40,-32
A,-63,-55,-47,-39


In [18]:
T

,_,G,T,C
_,,I,I,I
T,D,R,M,I
A,D,R,R,R
C,D,D,R,M
G,D,M,D,R
N,D,R,R,R
C,D,D,D,M
A,D,D,D,D
G,D,M,D,D
A,D,D,D,D
